# ANLY 515 — Risk Modeling and Optimization of a Multi-Sector Equity Portfolio

This notebook implements a minimal, clean experiment that runs top-to-bottom with no unnecessary complexity.


## 1) Environment & Setup

Install and import only the required libraries, set a seed, and silence warnings.


In [ ]:
# Install required packages
!pip install -q numpy pandas matplotlib seaborn yfinance scipy


In [ ]:
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy import stats
from scipy.optimize import minimize

# Silence warnings for clean output
warnings.filterwarnings("ignore")

# Reproducibility
SEED = 319302
np.random.seed(SEED)

# Plot styling
plt.rcParams["figure.figsize"] = (10, 5)


## 2) Data Collection

Download daily adjusted close prices from Yahoo Finance and clean missing values.


In [ ]:
# Sector definitions and tickers
sectors = {
    "Technology": ["AAPL", "MSFT"],
    "Financials": ["JPM", "BAC"],
    "Healthcare": ["JNJ", "PFE"],
}

all_tickers = [t for tickers in sectors.values() for t in tickers]
start_date = "2020-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")

# Download adjusted close prices
prices = yf.download(
    all_tickers,
    start=start_date,
    end=end_date,
    auto_adjust=False,
    progress=False,
)["Adj Close"]

# Ensure DataFrame
if isinstance(prices, pd.Series):
    prices = prices.to_frame(name=all_tickers[0])

# Drop missing values
prices = prices.dropna()

# Basic validation
assert not prices.empty, "Price data is empty. Check ticker symbols or network access."
assert prices.shape[1] == 6, "Expected 6 assets after download."

print(f"Downloaded price data: {prices.shape[0]} rows x {prices.shape[1]} columns")
prices.head()


## 3) Return Computation

Compute daily log returns for each asset.


In [ ]:
log_returns = np.log(prices / prices.shift(1)).dropna()

assert not log_returns.empty, "Log returns are empty after computation."

print(f"Log returns: {log_returns.shape[0]} rows x {log_returns.shape[1]} columns")
log_returns.head()


## 4) Descriptive Risk Statistics

Compute mean, volatility, and correlation for assets, sectors, and the total portfolio.


In [ ]:
# Asset-level stats
asset_stats = pd.DataFrame({
    "mean": log_returns.mean(),
    "volatility": log_returns.std(),
})
print("Asset-level statistics")
display(asset_stats)

# Sector portfolios (equal-weight within sector)
sector_returns = pd.DataFrame({
    sector: log_returns[tickers].mean(axis=1)
    for sector, tickers in sectors.items()
})

# Total portfolio (equal-weight across all assets)
portfolio_return = log_returns.mean(axis=1)

portfolio_stats = pd.DataFrame({
    "mean": sector_returns.join(portfolio_return.rename("Total")).mean(),
    "volatility": sector_returns.join(portfolio_return.rename("Total")).std(),
})

print("Sector and total portfolio statistics")
display(portfolio_stats)

# Correlation heatmap (assets)
correlation = log_returns.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Asset Correlation Matrix")
plt.tight_layout()
plt.show()


## 5) Value at Risk (VaR)

Compute 95% historical and parametric (normal) VaR for sector portfolios and total portfolio.


In [ ]:
def historical_var(series, alpha=0.95):
    return -np.quantile(series, 1 - alpha)


def parametric_var(series, alpha=0.95):
    mu = series.mean()
    sigma = series.std()
    z = stats.norm.ppf(1 - alpha)
    return -(mu + sigma * z)


portfolios = sector_returns.copy()
portfolios["Total"] = portfolio_return

var_rows = []
for name in portfolios.columns:
    s = portfolios[name]
    var_rows.append({
        "portfolio": name,
        "VaR_hist_95": historical_var(s, 0.95),
        "VaR_norm_95": parametric_var(s, 0.95),
    })

var_table = pd.DataFrame(var_rows).set_index("portfolio")
print("95% VaR (historical and normal)")
display(var_table)


## 6) Conditional Value at Risk (CVaR)

Compute 95% historical CVaR for sector portfolios and total portfolio.


In [ ]:
def historical_cvar(series, alpha=0.95):
    q = np.quantile(series, 1 - alpha)
    tail = series[series <= q]
    return -tail.mean()

cvar_rows = []
for name in portfolios.columns:
    s = portfolios[name]
    cvar_rows.append({
        "portfolio": name,
        "CVaR_hist_95": historical_cvar(s, 0.95),
    })

cvar_table = pd.DataFrame(cvar_rows).set_index("portfolio")
print("95% CVaR (historical)")
display(cvar_table)


## 7) Monte Carlo Simulation

Simulate 10,000 portfolio returns using a multivariate normal model and plot VaR/CVaR.


In [ ]:
mu = log_returns.mean().values
cov = log_returns.cov().values
n_sims = 10_000

weights_equal = np.ones(len(all_tickers)) / len(all_tickers)

simulated = np.random.multivariate_normal(mu, cov, size=n_sims)
sim_portfolio = simulated @ weights_equal

var_return = np.quantile(sim_portfolio, 0.05)
cvar_return = sim_portfolio[sim_portfolio <= var_return].mean()

plt.figure(figsize=(10, 5))
plt.hist(sim_portfolio, bins=60, color="steelblue", alpha=0.7)
plt.axvline(var_return, color="red", linestyle="--", label="VaR 95%")
plt.axvline(cvar_return, color="black", linestyle=":", label="CVaR 95%")
plt.title("Simulated Portfolio Return Distribution")
plt.xlabel("Daily Return")
plt.ylabel("Frequency")
plt.legend()
plt.tight_layout()
plt.show()


## 8) Portfolio Optimization (Simple)

Minimize portfolio variance with long-only weights and compare to equal-weight risk.


In [ ]:
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

n_assets = len(all_tickers)

# Constraints: weights sum to 1, no shorting
constraints = ({"type": "eq", "fun": lambda w: np.sum(w) - 1})
bounds = [(0.0, 1.0) for _ in range(n_assets)]

initial = np.ones(n_assets) / n_assets

result = minimize(
    portfolio_variance,
    initial,
    args=(cov,),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
)

assert result.success, f"Optimization failed: {result.message}"

weights_opt = result.x

risk_equal = np.sqrt(portfolio_variance(initial, cov))
risk_opt = np.sqrt(portfolio_variance(weights_opt, cov))

risk_compare = pd.DataFrame({
    "portfolio": ["Equal-weight", "Optimized"],
    "volatility": [risk_equal, risk_opt],
}).set_index("portfolio")

print("Equal-weight vs optimized portfolio risk")
display(risk_compare)

weights_table = pd.DataFrame({
    "Equal-weight": initial,
    "Optimized": weights_opt,
}, index=all_tickers)

print("Optimized weights")
display(weights_table)


## 9) Sector Risk Attribution (Simple)

Compute each sector’s contribution to total portfolio variance (equal-weight portfolio).


In [ ]:
# Asset-level variance contributions for equal-weight portfolio
port_var = portfolio_variance(initial, cov)
marginal = cov @ initial
asset_contrib = initial * marginal / port_var

contrib_table = pd.DataFrame({
    "asset": all_tickers,
    "variance_contribution": asset_contrib,
})

# Map asset contributions to sector totals
sector_contrib = {}
for sector, tickers in sectors.items():
    sector_contrib[sector] = contrib_table[contrib_table["asset"].isin(tickers)]["variance_contribution"].sum()

sector_contrib_table = pd.DataFrame.from_dict(sector_contrib, orient="index", columns=["variance_contribution"])

print("Sector variance contribution (equal-weight portfolio)")
display(sector_contrib_table)
